## Import Libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url = "https://www.nicd.ac.za/covid-19-update-57/"
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

## Check no. of columns

In [3]:
[len(T) for T in tr_elements[:12]]

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

## Check column titles

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Province"
2:"New cases on 15 May2020"
3:"Total cases for 15 May2020"
4:"Percentage total"


## Extract numerical data

In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 4, the //tr data is not from our table 
    if len(T)!=4:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

## Create dataframe

In [6]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df

,Province,New cases on 15 May2020,Total cases for 15 May2020,Percentage total
0,Eastern Cape,93,1662,"12,3"
1,Free State,6,151,"1,1"
2,Gauteng,75,2210,"16,3"
3,KwaZulu-Natal,38,1482,"11,0"
4,Limpopo,2,57,"0,4"
5,Mpumalanga,0,67,"0,5"
6,North West,5,63,"0,5"
7,Northern Cape,3,34,"0,3"
8,Western Cape,563,7798,"57,7"
9,Unknown,0,0,"0,0"


## Clean dataframe

In [7]:
# remove Unknown and Total columns
df = df[df.Province != 'Unknown']
df = df[df.Province != 'Total']

# drop the last column
df=df.drop(columns=['Percentage total'])

# Create date df

In [9]:
date = ['2020-05-15' for i in range(len(df))]

date_df = pd.DataFrame(date, columns=['date'])

## Merge date df to main df

In [11]:
df = date_df.join(df, how='left')

## Rename columns

In [12]:
df = df.rename(columns={"New cases on 15 May2020": "New cases", "Total cases for 15 May2020": "Confirmed"})

In [13]:
df

,date,Province,New cases,Confirmed
0,2020-05-15,Eastern Cape,93,1662
1,2020-05-15,Free State,6,151
2,2020-05-15,Gauteng,75,2210
3,2020-05-15,KwaZulu-Natal,38,1482
4,2020-05-15,Limpopo,2,57
5,2020-05-15,Mpumalanga,0,67
6,2020-05-15,North West,5,63
7,2020-05-15,Northern Cape,3,34
8,2020-05-15,Western Cape,563,7798


## To add deaths and recoveries

In [14]:
[len(T) for T in tr_elements[16:]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [15]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[16]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Province"
2:"Deaths"
3:"Recoveries"


In [16]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(17,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [17]:
Dict={title:column for (title,column) in col}
df2=pd.DataFrame(Dict)
df2

,Province,Deaths,Recoveries
0,Eastern Cape,31,643
1,Free State,6,108
2,Gauteng,24,1552
3,KwaZulu Natal,45,795
4,Limpopo,3,36
5,Mpumalanga,0,44
6,North West,1,28
7,Northern Cape,0,20
8,Western Cape,137,2857
9,Total,247,6083


In [18]:
df2 = df2[df2.Province != 'Total']

In [19]:
df2 = df2.drop(columns=['Province'])

In [21]:
df = df.join(df2)

In [22]:
df

,date,Province,New cases,Confirmed,Deaths,Recoveries
0,2020-05-15,Eastern Cape,93,1662,31,643
1,2020-05-15,Free State,6,151,6,108
2,2020-05-15,Gauteng,75,2210,24,1552
3,2020-05-15,KwaZulu-Natal,38,1482,45,795
4,2020-05-15,Limpopo,2,57,3,36
5,2020-05-15,Mpumalanga,0,67,0,44
6,2020-05-15,North West,5,63,1,28
7,2020-05-15,Northern Cape,3,34,0,20
8,2020-05-15,Western Cape,563,7798,137,2857


In [23]:
df.groupby('date').sum()

,New cases,Confirmed,Deaths,Recoveries
date,,,,
2020-05-15,785,13524,247,6083
